https://huggingface.co/docs/transformers/tasks/language_modeling#language-modeling

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

In [2]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import DataLoader

Using TensorFlow backend.


## 训练

In [ ]:
lines = open("data/after_preprocess/all_text.txt",'r').readlines()

In [ ]:
block_text_size = 128
all_seg_text_list = []
for line in lines:
    words = line.split(" ")
    total_length = len(words)
    total_length = (total_length // block_text_size) * block_text_size
    result = [" ".join(words[i : i + block_text_size]) for i in range(0, total_length, block_text_size)]
    all_seg_text_list.extend(result)

In [ ]:
len(all_seg_text_list)

In [ ]:
all_seg_text_list[:20]

In [ ]:
train_text_list,test_text_list = train_test_split(all_seg_text_list,test_size=0.1)
len(train_text_list),len(test_text_list)

In [ ]:
import torch
class essayDataset(torch.utils.data.Dataset):
    def __init__(self, datas):
        self.datas = datas

    def __len__(self):
        return len(self.datas)

    def __getitem__(self, item):
        one = self.datas[item]
        return one

In [ ]:
train_dataset = essayDataset(train_text_list)
test_dataset = essayDataset(test_text_list)

In [ ]:
MAX_SENT_LEN = 256
def collate(text_list):
    batch_input, batch_label = [], []
    for text in text_list:
        input_tokens = tokenizer.convert_tokens_to_ids(
            tokenizer.tokenize(text))
        label_one = input_tokens.copy()
        if len(input_tokens) > MAX_SENT_LEN:
            input_tokens = input_tokens[:MAX_SENT_LEN]
            label_one = label_one[:MAX_SENT_LEN]
        batch_input.append(input_tokens)
        batch_label.append(label_one)
    # dynamice padding
    max_len = max(len(x) for x in batch_input)
    batch_input = pad_sequences(batch_input, maxlen=max_len, dtype="long",
                                value=tokenizer.convert_tokens_to_ids('[PAD]'), padding="post")
    batch_label = pad_sequences(batch_label, maxlen=max_len, dtype="long",
                                value=-100, padding="post")
    
    input_js = {"input_ids":torch.LongTensor(batch_input),"labels":torch.LongTensor(batch_label)}
    return input_js

In [ ]:
from transformers import AutoTokenizer,AutoModel
# model_name = "EleutherAI/gpt-neo-1.3B"
model_name = "EleutherAI/gpt-j-6B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [ ]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained(model_name,cache_dir='./cache')

In [ ]:
model.resize_token_embeddings(len(tokenizer))

In [ ]:
batch_size = 1

In [ ]:
save_dir = 'model/task2/gpt-j-6B/lr-1e-5/'

In [ ]:
from transformers import TrainerCallback
import os
class SaveBestCallback(TrainerCallback):
    def __init__(self, save_dir):
        self.save_dir = save_dir
        self.best_result = 1000

    def on_evaluate(self, args, state, control, metrics, **kwargs):
        metric_to_check = args.metric_for_best_model
        if not metric_to_check.startswith("eval_"):
            metric_to_check = f"eval_{metric_to_check}"
            
        metric_value = metrics.get(metric_to_check)
        if metric_value < self.best_result:
            self.best_result = metric_value
            print("metric_value is {}".format(metric_value))
            tokenizer.save_pretrained(self.save_dir)
            model.save_pretrained(self.save_dir)

In [ ]:
training_args = TrainingArguments(
    output_dir="./model/task2/tmp",
    learning_rate=1e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=40,
    metric_for_best_model='loss',
    save_total_limit=1,
    eval_steps=50,
    evaluation_strategy='steps',
    fp16=True

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=collate,
    callbacks = [SaveBestCallback(save_dir)]
)

trainer.train()

In [ ]:
model = model.from_pretrained(save_dir)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=collate,
    callbacks = [SaveBestCallback(save_dir)]
)

In [ ]:
trainer.evaluate(test_dataset)

In [ ]:
sentence = "Okay, you think you're ready to hit the water? I guess. Don't worry, you'll be a natural. Okay. Whoo hoo! "
input_ids = tokenizer(sentence, return_tensors="pt").input_ids.to(model.device)

In [ ]:
outputs = model.generate(input_ids, do_sample=True, max_length=200)
tokenizer.batch_decode(outputs, skip_special_tokens=True)

## 推理

In [3]:
import pickle
import torch
from tqdm import tqdm_notebook
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

end_tokens = ['.','!','?']
def split_sent(text):
    sents = [x.strip()+'.' if x[-1] not in end_tokens else x.strip() for x in text.split(".") if len(x)!=0]
    return sents

In [4]:
model_name = 'model/task2/gpt-neo-1.3B/lr-1e-5/'

In [5]:
from transformers import AutoTokenizer,AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [6]:
model = AutoModelForCausalLM.from_pretrained(model_name)
model = model.to(device)

In [24]:
def generate_text(input_text,model,max_length=300,num_return=5):
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(model.device)
    with torch.no_grad():
        outputs = model.generate(input_ids, do_sample=True, max_length=max_length,use_cache=True,num_return_sequences=num_return)
    output_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    gen_text_list = []
    for text in output_texts:
        gen_text = split_sent(text.split(input_text)[1].strip())[0]
        gen_text_list.append(gen_text)
    return output_texts,gen_text_list

In [25]:
text = "Hello, my name is Jiahao Chen."
output_texts,gen_text_list = generate_text(text,model)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [26]:
gen_text_list

["Oh, you're the famous magician.",
 'Welcome to the P.',
 'I have an idea for the video diary: A little fun for the diary.',
 'I want to make some money.',
 "And I'm a security officer here at N."]

In [34]:
input_js_list = pickle.load(open('data/after_preprocess/task2/input_js_list.pkl','rb'))

In [35]:
for input_js in tqdm_notebook(input_js_list):
    input_text = " ".join(" ".join(input_js['history_text']).split(" ")[-200:])
    output_texts,gen_text_list = [],[]
    for _ in range(100):
        sub_output_texts,sub_gen_text_list = generate_text(input_text,model,num_return=30)
        output_texts.extend(sub_output_texts)
        gen_text_list.extend(sub_gen_text_list)
    input_js['output_texts'] = output_texts
    input_js['gen_text_list'] = gen_text_list

/tmp/ipykernel_419/3194433971.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for input_js in tqdm_notebook(input_js_list):


  0%|          | 0/39 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable resul

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable resul

...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable resul

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable resul

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable resul

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable resul

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable resul

In [36]:
with open('data/after_preprocess/task2/input_js_list_gen.pkl','wb') as f:
    pickle.dump(input_js_list,f)